# Final Exam

## FINM 37400 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is 180 points.
* You have 180 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.
Final Exam

Submission
* You will upload your solution to the Final Exam assignment on Canvas, where you downloaded this.
* Your submission should be readable, (the graders can understand your answers,) and it should include all code used in your analysis in a file format that the code can be executed. (ie. .ipynb preferred, .pdf is unacceptable.)

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation, (only for this exam.)

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:

### Sections 2, 3, 4
* `treasury_ts_2020-08-15.xlsx`

If you get stuck on 2.2, you can replace that answer with the data in 
* `simplified_spot_discounts_2022-12-30.xlsx`

But if you solve 2.2, then you'll have no use/need of this data.

### Section 5
* `treasury_ts_2020-08-15.xlsx`

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 50     |
| 2       | 25     |
| 3       | 35     |
| 4       | 20     |
| 5       | 50     |

# 1. Short Answer

### No Data Needed

These problem does not require any data file. Rather, analyze the situation conceptually, based on the information below. 

A few questions may require simple arithmetic.

## 1.1 (5pts)

True or False: The spot rate prices a particular security, and YTM prices any cashflow at a particular maturity.

Explain your answer.

## 1.2 (5pts)

Suppose the duration of a bond is 7.0. The current price of the bond is 99.8 and the YTM of the bond is 3.27\%. 

The spot-rate curve moves up in a parallel shift by 10bps. 

* What is the approximate price of the bond after the shift?

* Do you suspect this approximation is an underestimate or overestimate? 

## 1.3 (5pts)

Are duration-based approximations of bond prices more accurate for a Treasury with high coupon rate or low coupon rate, (holding other things, like time-to-maturity, equal?)

## 1.4 (5pts)

Which tend to be less smooth: spot curves or forward curves? 

Explain.

## 1.5 (5pts)

True or False: Bond prices and returns have nearly zero serial correlation.

Explain your answer.

## 1.6 (5pts)

As of Dec 2022, the yield curve is "inverted"?

Does this inversion mathematically imply that expected future spot rates will decline, or is there potentially another explanation?

## 1.7 (5pts)

Suppose we want to estimate a discount curve from a set of Treasuries.

If there are dates where multiple treasuries pay a coupon yet none expire, which of our estimation methods have trouble?
* Bootstrap
* OLS
* Nelson-Siegel

Explain.

## 1.8 (5pts)

Suppose that yesterday you built a position
* long a 10-year floating-rate note that resets quarterly
* short a 5-year floating-rate note that resets annually

Is your duration positive or negative? How about four years from today?

Explain.

## 1.9 (10pts)

### Situation
Suppose it is Aug 15, 2022. We observe the following annual rates, (semiannually compounded):
* 7-year T-note (newly issued) has a price of par, with a coupon of 3.50%.
* 7-year swap (semi-annual, SOFR) has a swap rate of 3.00%.

*Assume the floating leg equals the repo rate and timing.*

### 1.9.1
**Conceptually**, explain the short and long positions you wouldl take (on notional of \\$100 million) in order to try to profit on this spread **diverging** further, meaning the treasury rate going higher relative to the swap rate.

### 1.9.2

Suppose that it is February 15, 2023, immediately after the swap has reset and the cashflows of the trade were settled.

In the market, we observe the following:
* The 5-year swap rate is 3.50%.
* The 5-year Treasury has a yield-to-maturity of 4.25%. 

**Conceptually**, How has the value changed between over the six months of each component of the trade?
* swap 
* T-note

***

# 2 Basic Pricing

Use the selected treasury quotes in the file, `selected_treasury_quotes_quotes_2022-12-30.xlsx`.

Data is provided on 10 Treasury issues which mature every six months over the next five years.

*You are welcome to assume that the issues mature in exactly half-year increments, notwithstanding that in actual trading they may have day-count issues causing them to be slightly above/below half-year increments.*

## 2.1 (5pts)

Calculate the cashflow matrix for the treasury issues.

Display the table.

In [1]:
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import optimize as opt
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from functions import *

In [2]:
df=pd.read_excel("../data/selected_treasury_quotes_2022-12-30.xlsx")
df

,KYTREASNO,issue date,maturity date,maturity,coupon rate,price,ytm
0,206990,2016-06-30,2023-06-30,0.498289,1.375,98.460938,0.044952
1,207047,2016-12-31,2023-12-31,1.002053,2.250,97.585938,0.046915
2,207374,2019-06-30,2024-06-30,1.500342,1.750,95.851562,0.045942
3,207166,2017-12-31,2024-12-31,2.004107,2.250,95.859375,0.043822
4,207224,2018-06-30,2025-06-30,2.499658,2.750,96.429688,0.042295
5,207652,2020-12-31,2025-12-31,3.003422,0.375,89.378906,0.041313
6,207731,2021-06-30,2026-06-30,3.498973,0.875,89.460938,0.041003
7,207821,2021-12-31,2026-12-31,4.002738,1.250,89.562500,0.040628
8,207561,2020-06-30,2027-06-30,4.498289,0.500,85.562500,0.040027
9,207989,2022-12-31,2027-12-31,5.002053,3.875,99.417969,0.039634


In [3]:
df['yield_test']=df.apply(lambda x: get_yield(x['price'],x['maturity'],x['coupon rate'], 2), axis=1)*365.25
df

,KYTREASNO,issue date,maturity date,maturity,coupon rate,price,ytm,yield_test
0,206990,2016-06-30,2023-06-30,0.498289,1.375,98.460938,0.044952,0.045384
1,207047,2016-12-31,2023-12-31,1.002053,2.250,97.585938,0.046915,0.059351
2,207374,2019-06-30,2024-06-30,1.500342,1.750,95.851562,0.045942,0.052760
3,207166,2017-12-31,2024-12-31,2.004107,2.250,95.859375,0.043822,0.050403
4,207224,2018-06-30,2025-06-30,2.499658,2.750,96.429688,0.042295,0.042715
5,207652,2020-12-31,2025-12-31,3.003422,0.375,89.378906,0.041313,0.042456
6,207731,2021-06-30,2026-06-30,3.498973,0.875,89.460938,0.041003,0.041419
7,207821,2021-12-31,2026-12-31,4.002738,1.250,89.562500,0.040628,0.042861
8,207561,2020-06-30,2027-06-30,4.498289,0.500,85.562500,0.040027,0.040427
9,207989,2022-12-31,2027-12-31,5.002053,3.875,99.417969,0.039634,0.044404


In [4]:
C=C_matrix(df, '2022-12-30','maturity date','KYTREASNO','coupon rate',2)
C

,2023-06-30,2023-12-31,2024-06-30,2024-12-31,2025-06-30,2025-12-31,2026-06-30,2026-12-31,2027-06-30,2027-12-31
KYTREASNO,,,,,,,,,,
206990,100.6875,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
207047,1.1250,101.1250,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
207374,0.8750,0.8750,100.8750,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
207166,1.1250,1.1250,1.1250,101.1250,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
207224,1.3750,1.3750,1.3750,1.3750,101.3750,0.0000,0.0000,0.0000,0.0000,0.0000
207652,0.1875,0.1875,0.1875,0.1875,0.1875,100.1875,0.0000,0.0000,0.0000,0.0000
207731,0.4375,0.4375,0.4375,0.4375,0.4375,0.4375,100.4375,0.0000,0.0000,0.0000
207821,0.6250,0.6250,0.6250,0.6250,0.6250,0.6250,0.6250,100.6250,0.0000,0.0000
207561,0.2500,0.2500,0.2500,0.2500,0.2500,0.2500,0.2500,0.2500,100.2500,0.0000


## 2.2 (5pts)

Use the bootstrap (or OLS) to extract the spot discount factors from this data, over the maturity range of 0.5 to 5, in half-year increments.

### Note: If you couldn't extract spot discount factors...

Several problems below make use of the spot discounts on Dec 30, 2022.

**If you were unable to calculate the answer to 2.3,** feel free to use the simplified discount factors, $Z(t,T)$ provided in the file `simplified_spot_discounts_2022-12-30.xlsx`.

Note that the rates in this file are simplified and won't match the answer you'll get if you solve Problem 2.2 above, so don't try to check your answer with them.

If you are proceeding with the simplified rates rather than your estimates in 2.2, **state that clearly.**

And again, this is **just provided as a help for anyone who couldn't solve 2.2**, so that they can continue with 2.3-2.5, as well as other problems in the exam.

In [5]:
tprices=df.set_index('KYTREASNO',drop=True).loc[C.index]['price']
model=LinearRegression(fit_intercept=False)
model.fit(C,tprices)
res=pd.DataFrame({'discount_factor':model.coef_,'maturity':(pd.to_datetime(C.columns)-pd.to_datetime('2022-12-30')).days/365.25}).set_index('maturity',drop=True)
res

,discount_factor
maturity,
0.498289,0.977886
1.002053,0.954124
1.500342,0.933443
2.004107,0.916052
2.499658,0.899927
3.003422,0.883355
3.498973,0.866473
4.002738,0.850116
4.498289,0.835333


## 2.3 (5pts)

Report the 
* semi-annually-compounded spot discount **rates** derived from the discount factors above.
* continuously-compounded spot rates.

In [6]:
res['continuously']=continuous_from_discount(res['discount_factor'], res.index)
res['semiannual']=compounded_from_discount(res['discount_factor'], res.index, 2)
res

,discount_factor,continuously,semiannual
maturity,,,
0.498289,0.977886,0.044877,0.045384
1.002053,0.954124,0.046865,0.047419
1.500342,0.933443,0.045907,0.046437
2.004107,0.916052,0.043751,0.044233
2.499658,0.899927,0.042182,0.042630
3.003422,0.883355,0.041296,0.041725
3.498973,0.866473,0.040962,0.041384
4.002738,0.850116,0.040568,0.040982
4.498289,0.835333,0.039998,0.040401


## 2.4 (3pts)

Without any calculation, just looking at your spot rates from 2.3, is the long-end of the forward curve below or above the spot-rate curve? How do you know?

## 2.5 (7pts)
Calculate and report the continuously-compounded forward-rate curve, $f(t,T,T+1)$, for $0\le T \le 4$.

Also report the forward rate, $f(t,T_1,T_2)$, for $T_1=3, T_2=5$.

In [7]:
res['forward_factor']=res['discount_factor']/res['discount_factor'].shift(2)
res['forward_factor'].iloc[1]=res['discount_factor'].iloc[1]
res['forward_rate']=continuous_from_discount(res['forward_factor'],1)
res

,discount_factor,continuously,semiannual,forward_factor,forward_rate
maturity,,,,,
0.498289,0.977886,0.044877,0.045384,NaN,NaN
1.002053,0.954124,0.046865,0.047419,0.954124,0.046961
1.500342,0.933443,0.045907,0.046437,0.954551,0.046514
2.004107,0.916052,0.043751,0.044233,0.960097,0.040721
2.499658,0.899927,0.042182,0.042630,0.964095,0.036566
3.003422,0.883355,0.041296,0.041725,0.964307,0.036346
3.498973,0.866473,0.040962,0.041384,0.962825,0.037883
4.002738,0.850116,0.040568,0.040982,0.962372,0.038354
4.498289,0.835333,0.039998,0.040401,0.964062,0.036600


In [8]:
res.index=np.arange(0.5,5.5,0.5)
continuous_from_discount(res.loc[5,'discount_factor']/res.loc[3, 'discount_factor'], 2)


0.03659516740415847

In [9]:
res

,discount_factor,continuously,semiannual,forward_factor,forward_rate
0.5,0.977886,0.044877,0.045384,NaN,NaN
1.0,0.954124,0.046865,0.047419,0.954124,0.046961
1.5,0.933443,0.045907,0.046437,0.954551,0.046514
2.0,0.916052,0.043751,0.044233,0.960097,0.040721
2.5,0.899927,0.042182,0.042630,0.964095,0.036566
3.0,0.883355,0.041296,0.041725,0.964307,0.036346
3.5,0.866473,0.040962,0.041384,0.962825,0.037883
4.0,0.850116,0.040568,0.040982,0.962372,0.038354
4.5,0.835333,0.039998,0.040401,0.964062,0.036600
5.0,0.821011,0.039427,0.039819,0.965763,0.034836


***

# 3 An Amortized Bond

Suppose the Treasury announces a new type of bond, the **zero-coupon amortized bond** which works as follows:
* it pays the \\$100 face value evenly and annually over the life of the bond.
* it does not pay any additional coupon.

In particular, consider a five-year amortized bond:
* it pays \\$10 twice per year for the next 5 years.
* it does **not** pay any extra principal when it matures at 5 years--just the usual \\$10 payment.

## 3.1 (5pts)

Use the spot discount factors from Problem 2 to price this new security as of December 30, 2022.

In [10]:
price=np.sum(10*res['discount_factor'])
price

89.37721216287764

## 3.2 (5pts)

Calculate the YTM of this new **amortized bond**.

In [11]:
def bond_price_aux(yield_):
    global price
    tp=0
    for i in range(1,11):
        tp+=10*1/(1+yield_/2)**(i)
    return price-tp

ytm=opt.root(bond_price_aux, 0.02).x[0]
ytm

0.04191625993798047

## 3.3 (5pts)

Conceptually speaking, should the duration of this amortized bond be higher or lower than a typical zero-coupon, five-year treasury bond?

And its convexity?

## 3.4 (7pts)

Calculate the (Macauley) duration of this **amortized** bond.

In [12]:
weights=10*res['discount_factor']/price
duration=weights.dot(res.index)
duration

2.6709710451963096

## 3.5 (6pts)

Calculate the convexity of this amortized bond.

In [13]:
convexity=weights.dot(res.index**2)
convexity

9.196384783510787

## 3.6 (7pts)

Suppose you want to go long this amortized bond but hedge your duration by shorting the 5-year Treasury reported in `selected_treasury_quotes_quotes_2022-12-30.xlsx`.

For each long contract of the 5-year amortized, how many contracts should you short of the vanilla 5-year Treasury?

In [14]:
df['duration']=df.apply(lambda x: macDuration(x['ytm'],x['coupon rate'],x['maturity'],2), axis=1)

In [15]:
-duration*price/(df['price'].iloc[-1]*df['duration'].iloc[-1])


-0.5328036873690204

***

# 4 Expectations Hypothesis

Continue using the rates from Dec 30, 2022 introduced in Problem 2 above.

## 4.1 (5pts)

Use the Expectations Hypothesis to get a forecast the one-year spot rate that will be available in December of the following four years, (2023, 2024, 2025, 2026.)


In [16]:
res.loc[[1,2,3,4,5]]

,discount_factor,continuously,semiannual,forward_factor,forward_rate
1.0,0.954124,0.046865,0.047419,0.954124,0.046961
2.0,0.916052,0.043751,0.044233,0.960097,0.040721
3.0,0.883355,0.041296,0.041725,0.964307,0.036346
4.0,0.850116,0.040568,0.040982,0.962372,0.038354
5.0,0.821011,0.039427,0.039819,0.965763,0.034836



## 4.2 (5pts)

Based on evidence we reviewed of the Expectations Hypothesis, do you expect the forecasts above are too high or low? Explain.


The forecasts are likely too high.

In W.5., we saw evidence that forward rates are useful as forecasts, but that they are (on average) higher than the future one-year spot rates. This was one of the pieces of evidence that the Expectations Hypothesis is false.


## 4.3 (5pts)

Suppose that the following month, January 2023, we observe that the forward rates have increased. Is it likely that the future spot rates (in Jan 2023, 2024, 2025, 2026) will increase as well? Explain.


Yes, **on average** they increase, but it is not **likely**.

In W.5., in the Dynamic Tests of the EH, Fact 2 ran a regression showing that innovations to the forward rate are positively correlated with changes to the future spot (positive betas) but that it is an extremely weak relationship (r-squared near zero.)


## 4.4 (5pts)

As of December 2022, describe (conceptually) the "carry trade" you would recommend an investor implement..

We saw that an upward sloping spot curve predicts profits to going long the long-term bond and shorting the short-term bond, due to the failure of the Expectations Hypothesis. (See HW 5).

**Thus, given the downward sloping spot curve in Dec 2022, the appropriate carry trade would be to set up the following position:**
* short long-term (i.e. 5-year) bonds
* long short-term (i.e. 1-year) bonds

See the Dynamic Tests of the EH in W.5. for more. This trade carries interest-rate risk, but those tests indicate it has positive expected PnL.

***

# 5 Long-Short Convergence Trade

Consider a long-short convergence trade on `2019-08-29`.

* Both securities mature on **2020-08-15**.
* One is a 30-year **bond** and the other is a 10-year **note**.

Find data needed for the analysis in `treasury_ts_2020-08-15.xlsx`.

Note:
* Use the YTM as given in the field `TDYLD`. Multiply by 365.25 to convert it to an annual yield.
* Use the (Macauley) duration given in `TDDURATN`, divided by 365.25 to annualize it.

## 5.1 (10pts)

Display a table showing the two securities as of `2019-08-29`:
* issue date
* maturity date
* coupon rate
* clean price (taken as average of bid and ask)
* dirty price
* duration (Macauley)
* modified duration
* ytm

In [17]:
data=pd.read_excel("../data/treasury_ts_2020-08-15.xlsx")
data

,Unnamed: 0,0,1
0,kytreasno,204074,206169
1,kycrspid,20200815.108750,20200815.202620
2,crspid,20200815.108750,20200815.202620
3,tcusip,912810EG,912828NT
4,tdatdt,1990-08-15 00:00:00,2010-08-15 00:00:00
5,tmatdt,2020-08-15 00:00:00,2020-08-15 00:00:00
6,iwhy,1,1
7,tcouprt,8.75,2.625
8,tnippy,2,2
9,tvalfc,4.375,1.3125


In [18]:
database=pd.read_excel("../data/treasury_ts_2020-08-15.xlsx", sheet_name='database')
database['caldt']=pd.to_datetime(database['caldt'], format='%Y%m%d')
data.loc[4][1:]=pd.to_datetime(data.loc[4][1:], format='%Y%m%d')
data.loc[10][1:]=pd.to_datetime(data.loc[10][1:], format='%Y%m%d')
ttbonds=database[database['caldt']=='2019-08-29'].copy()
tbonds=ttbonds.iloc[:,:].copy()
tbonds

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
5778,204074,2.020082e+07,2019-08-29,106.71875,106.742188,106.730469,M,I,0.332880,-0.000216,0.000045,344.619935,5118.0,17059.0,0.0,NaN,NaN
8034,206169,2.020082e+07,2019-08-29,100.78125,100.804688,100.792969,M,I,0.099864,-0.000278,0.000049,349.651903,34231.0,67850.0,0.0,NaN,NaN


In [19]:
database

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,204074,2.020082e+07,1993-04-13,122.593750,122.656250,122.625000,M,R,1.377762,0.001204,0.000186,4387.142817,21009.0,21419.0,0.0,NaN,NaN
1,204074,2.020082e+07,1990-08-10,99.562500,99.625000,99.593750,M,R,0.000000,NaN,0.000235,4021.731911,NaN,NaN,0.0,NaN,NaN
2,204074,2.020082e+07,1990-08-13,99.187500,99.250000,99.218750,M,R,0.000000,-0.003765,0.000236,4006.695342,NaN,NaN,0.0,NaN,NaN
3,204074,2.020082e+07,1990-08-14,99.656250,99.718750,99.687500,M,R,0.000000,0.004724,0.000235,4017.732737,NaN,NaN,0.0,NaN,NaN
4,204074,2.020082e+07,1990-08-15,99.843750,99.906250,99.875000,M,R,0.000000,0.001881,0.000235,4021.154173,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10001,206169,2.020082e+07,2017-07-07,103.039062,103.070312,103.054688,M,I,1.029696,0.000070,0.000044,1087.375558,34231.0,67850.0,0.0,NaN,NaN
10002,206169,2.020082e+07,2017-07-10,103.039062,103.054688,103.046875,M,I,1.051450,0.000134,0.000044,1084.375623,34231.0,67850.0,0.0,NaN,NaN
10003,206169,2.020082e+07,2017-07-11,103.093750,103.125000,103.109375,M,I,1.058702,0.000670,0.000043,1083.396165,34231.0,67850.0,0.0,NaN,NaN
10004,206169,2.020082e+07,2017-07-12,103.203125,103.234375,103.218750,M,I,1.065953,0.001120,0.000042,1082.431479,34231.0,67850.0,0.0,NaN,NaN


In [20]:
bonds=pd.DataFrame()
bonds['id']=data.loc[1][1:]
bonds['issue_date']=pd.to_datetime(data.loc[4][1:])
bonds['maturity_date']=pd.to_datetime(data.loc[5][1:])
bonds['coupon_rate']=data.loc[7][1:]
bonds['clean_price']=((tbonds['tdbid']+tbonds['tdask'])/2).values
bonds['dirty_price']=bonds['clean_price']+(tbonds['tdaccint']).values
bonds['duration']=(tbonds['tdduratn']).values/365.25
bonds['ytm']=(tbonds['tdyld']).values*365.25
bonds['mod_duration']=bonds['duration']/(1+(bonds['ytm']/2))
bonds

,id,issue_date,maturity_date,coupon_rate,clean_price,dirty_price,duration,ytm,mod_duration
0,20200815.108750,1990-08-15,2020-08-15,8.75,106.730469,107.063349,0.943518,0.016565,0.935767
1,20200815.202620,2010-08-15,2020-08-15,2.625,100.792969,100.892833,0.957295,0.017782,0.948859


## 5.2 (5pts)

On this date, `2019-08-29`, the YTM spread is at a 5-year low.

Calculate the YTM spread (bond YTM minus note YTM).
* Report this number.

Explain conceptually what trade might make sense if we believe this spread will converge back to zero.
* Which security will you go long and which will you go short?

In [25]:
bonds['ytm'].iloc[0]-bonds['ytm'].iloc[1]

-0.0012162607418438297

note higher yield = it should decrease = decreasing yield = price increase

long note, short bond

## 5.3 (5pts)
Describe conceptually the risk in this trade.
* What risks are present in this trade? 
* Is it an arbitrage? 

Answer both over the next month and over the next year, until maturity.

## 5.4 (10pts)

Suppose you use \\$1 million of equity capital on the long position, leveraged 50x to buy \\$50 million of the long position. Assume any short position will also be levered 50x. 

*You do not need to worry about the details of this financing. One could view this as a 2\% capital requirement on both the long and short positions, possibly due to haircuts and brokerage requirements. There is no difference between financing rates on the long and short positions, (so no need to consider repo rate vs brokerage account interest rate.)*

**Calculate...**
* the number of long contracts 
* the number of short contracts such that the trade is initialized with zero duration
* the dollar value of assets and of equity in each side of the trade.

In calculating the number of contracts, assume face value of \\$100 each.

In [26]:
nlong=50/bonds['dirty_price'].iloc[1]
ddur_long=bonds['duration'].iloc[1]*bonds['dirty_price'].iloc[1]
ddur_short=bonds['duration'].iloc[0]*bonds['dirty_price'].iloc[0]
nshort=-nlong*ddur_long/ddur_short
nshort

-0.4738323182931589

In [27]:
print(nlong)
print(nshort)
print(50)
print(nshort*bonds['dirty_price'].iloc[0])

0.495575340413462
-0.4738323182931589
50
-50.73007494845553


## 5.5 (10pts)

Using the concept of modified duration, how much profit or loss (PnL) would you expect to make for every basis point of convergence in the spread? 

Specifically, assume the convergence is symmetric: the long position's ytm goes down 0.5bp and the short position's ytm goes up 0.5bp. 

**Using this approximation, specify the PnL of the long position, the short position, and the net total for a 1bp convergence.**

In [30]:
nlong*bonds['mod_duration'].iloc[1]*0.00005*bonds['dirty_price'].iloc[1]*1000000

2372.1466299551644

## 5.6 (5pts)

Calculate the profit (or loss) on the position as of `2019-10-01`.

Show this separately for the long position, the short position, and the net.

Note that no coupon is paid between the trade being set up and this date; thus, the PnL is simply a matter of the position's prices.

## 5.7 (5pts)

Does the actual PnL above make sense in light of the duration-based PnL-per-bp calculated above? Give two reasons these numbers may not be aligned.

***